# Natural Gas Price Prediction Model Development

This notebook demonstrates the development process of our natural gas price prediction model, including data analysis, model selection, and validation.

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn')

# Load data
df = pd.read_csv('../data/raw/Nat_Gas.csv')
df['Dates'] = pd.to_datetime(df['Dates'])
df.set_index('Dates', inplace=True)

print("Dataset Overview:")
print(f"Time Range: {df.index.min()} to {df.index.max()}")
print(f"Number of observations: {len(df)}")
print("\nBasic Statistics:")
print(df.describe())

## 2. Time Series Analysis

### 2.1 Initial Data Visualization

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Prices'])
plt.title('Natural Gas Price History')
plt.xlabel('Date')
plt.ylabel('Price')
plt.xticks(rotation=45)
plt.show()

### 2.2 Seasonal Decomposition

In [ ]:
# Perform seasonal decomposition
decomposition = seasonal_decompose(df['Prices'], period=12)

# Plot components
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(12, 10))

decomposition.observed.plot(ax=ax1)
ax1.set_title('Observed')

decomposition.trend.plot(ax=ax2)
ax2.set_title('Trend')

decomposition.seasonal.plot(ax=ax3)
ax3.set_title('Seasonal')

decomposition.resid.plot(ax=ax4)
ax4.set_title('Residual')

plt.tight_layout()
plt.show()

### 2.3 Stationarity Testing

In [ ]:
def test_stationarity(series):
    result = adfuller(series)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

print("Testing stationarity of original series:")
test_stationarity(df['Prices'])

## 3. Model Development

### 3.1 Data Preparation

In [ ]:
# Split data for training and testing
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

print(f"Training data: {len(train_data)} observations")
print(f"Testing data: {len(test_data)} observations")

### 3.2 Model Fitting

In [ ]:
# Fit SARIMA model
model = SARIMAX(train_data['Prices'],
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 12))

fitted_model = model.fit(disp=False)
print(fitted_model.summary())

### 3.3 Model Evaluation

In [ ]:
# Generate predictions
predictions = fitted_model.get_prediction(start=test_data.index[0],
                                       end=test_data.index[-1])
predicted_mean = predictions.predicted_mean

# Calculate metrics
rmse = np.sqrt(mean_squared_error(test_data['Prices'], predicted_mean))
mae = mean_absolute_error(test_data['Prices'], predicted_mean)
r2 = r2_score(test_data['Prices'], predicted_mean)

print("Model Performance Metrics:")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")

### 3.4 Visualization of Results

In [ ]:
# Plot predictions vs actual
plt.figure(figsize=(12, 6))
plt.plot(test_data.index, test_data['Prices'], label='Actual')
plt.plot(test_data.index, predicted_mean, label='Predicted')
plt.title('Model Predictions vs Actual Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Model Diagnostics

In [ ]:
# Plot residuals
residuals = test_data['Prices'] - predicted_mean

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Residuals over time
ax1.plot(test_data.index, residuals)
ax1.set_title('Residuals Over Time')
ax1.set_xlabel('Date')
ax1.set_ylabel('Residual')

# Residuals distribution
sns.histplot(residuals, kde=True, ax=ax2)
ax2.set_title('Residuals Distribution')
ax2.set_xlabel('Residual')

plt.tight_layout()
plt.show()

## 5. Future Predictions

In [ ]:
# Generate future predictions
future_dates = pd.date_range(start=df.index[-1], periods=12, freq='M')
future_predictions = fitted_model.forecast(steps=12)

# Plot historical data and predictions
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Prices'], label='Historical')
plt.plot(future_dates, future_predictions, '--', label='Forecast')
plt.title('Natural Gas Price Forecast')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()